In [39]:
# set required info

TEST_SET_KEY = 'GTSQA-317'
# VERSIONS = ["Kuiper2.0 Rel1"]


In [40]:
# Initialize script
from atlassian import Jira, Xray

JIRA_URL = "https://jira.analog.com"
API_TOKEN = "put your api key here"
PROJECT_KEY = "GTSQA"

jira = Jira(
    url=JIRA_URL,
    token=API_TOKEN
)

xray = Xray(
    url=JIRA_URL,
    token=API_TOKEN
)



In [41]:
# get test set information
test_set_key = TEST_SET_KEY
test_set_summary = jira.issue_field_value(test_set_key, "summary")
test_set_components = jira.issue_field_value(test_set_key, "components")

# get tests
test_set = xray.get_tests_with_test_set(test_set_key)
if len(test_set) == 0:
    raise Exception(f"{test_set} not found")

print(f"KEY: {test_set_key}")
print(f"Summary: {test_set_summary}")
print(f"Components: {test_set_components}")
print(f"Tests: {[test['key'] for test in test_set]}")

KEY: GTSQA-317
Summary: rpi4b-cn0552 tests
Components: [{'self': 'https://jira.analog.com/rest/api/2/component/68232', 'id': '68232', 'name': 'rpi4b'}]
Tests: ['GTSQA-165', 'GTSQA-173', 'GTSQA-182', 'GTSQA-183', 'GTSQA-185', 'GTSQA-184', 'GTSQA-187', 'GTSQA-186', 'GTSQA-175', 'GTSQA-318']


In [42]:
# Create a new test execution from test_set summary
summary = test_set_summary.split(" test")
if len(summary) > 0:
    summary = summary[0]

version_found = []
for version in jira.get_project_versions(PROJECT_KEY):
    if version["name"] in VERSIONS:
        version_found.append(version["name"])

def create_test_execution():
    issue_dict = {
        "project": {"key": PROJECT_KEY},
        "summary": f"Test {summary}",
        "issuetype": {"name": "Test Execution"},
        "description": f"Test Execution for {test_set_key} {test_set_summary}",
        "assignee": {"name": jira.myself()["key"]},
        # "reporter": {"name": "jiraUserName"},
        "versions": [{'name': version} for version in version_found],
        "components": [{"name": component['name']} for component in test_set_components],
        "labels": ["Kuiper2.0-2024_r2-pre-RC0"] 
    }

    print(issue_dict)

    # Create the issue
    test_exec = jira.issue_create(issue_dict)
    return test_exec["key"]

test_exec_key = create_test_execution()
print(test_exec_key)

# Link Test Cases to the Test Execution
# test_exec_key = "GTSQA-497"
print(xray.get_tests_with_test_execution(test_exec_key, detailed=True))

# Associate tests with the given test execution
xray.update_test_execution(test_exec_key, add=[test["key"] for test in test_set])

# Verify if tests were added
print(xray.get_tests_with_test_execution(test_exec_key, detailed=True))

{'project': {'key': 'GTSQA'}, 'summary': 'Test rpi4b-cn0552', 'issuetype': {'name': 'Test Execution'}, 'description': 'Test Execution for GTSQA-317 rpi4b-cn0552 tests', 'assignee': {'name': 'kpaller'}, 'versions': [{'name': 'Kuiper2.0 Rel1'}], 'components': [{'name': 'rpi4b'}], 'labels': ['Kuiper2.0-6-6']}
GTSQA-636
[]
[{'id': 458060, 'status': 'TODO', 'assignee': 'KPaller', 'defects': [], 'evidences': [], 'archived': False, 'key': 'GTSQA-165', 'rank': 1}, {'id': 458061, 'status': 'TODO', 'assignee': 'KPaller', 'defects': [], 'evidences': [], 'archived': False, 'key': 'GTSQA-173', 'rank': 2}, {'id': 458062, 'status': 'TODO', 'assignee': 'KPaller', 'defects': [], 'evidences': [], 'archived': False, 'key': 'GTSQA-182', 'rank': 3}, {'id': 458063, 'status': 'TODO', 'assignee': 'KPaller', 'defects': [], 'evidences': [], 'archived': False, 'key': 'GTSQA-183', 'rank': 4}, {'id': 458064, 'status': 'TODO', 'assignee': 'KPaller', 'defects': [], 'evidences': [], 'archived': False, 'key': 'GTSQA-1